# Asynchronous GraphQL Data Fetching data Mahasiswa
Notebook ini menunjukkan bagaimana kita bisa melakukan asynchronous data fetching dengan GraphQL SiCekCok untuk mengambil data mahasiswa menggunakan Python. Notebook ini menggunakan library `aiohttp` untuk melakukan HTTP request secara asynchronous dan `aiographql` untuk melakukan query ke GraphQL server.

### Langkah 0: Install Pustaka yang Diperlukan
Pada langkah Persiapan, kita menginstall pustaka `aiohttp` untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.

In [ ]:
%pip install aiohttp pandas

### Langkah 1: Mengimpor Pustaka yang Diperlukan
Pada langkah pertama, kita mengimpor pustaka `aiohttp` `asyncio`  untuk untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.


In [ ]:
import aiohttp
import asyncio
import pandas as pd

### Langkah 2: Mendefinisikan Fungsi untuk Query GraphQL
Kita mendefinisikan fungsi `query_graphql` yang mengambil parameter `nim` dan mengirimkan permintaan POST ke endpoint GraphQL yang ditentukan. Fungsi ini mengembalikan hasil dalam format JSON.

In [ ]:
async def query_graphql(session, nim):
    url = 'https://sicekcok.if.unismuh.ac.id/graphql'  # Ganti dengan URL endpoint GraphQL yang sesuai
    query = """
    query($nim: String!) {
      mahasiswa(nim: $nim) {
        nim
        kodeProdi
        angkatan
        nama
        jenisKelamin
        semesterAwal
        tahunAkademikLulus
        tanggalLulus
        lulus
        masaStudi
        khs {
          tahunAkademik
          ips
          sksSmt
          ipk
          sksTotal
          statusMahasiswa
        }
      }
    }
    """
    variables = {'nim': str(nim)}  # Pastikan NIM dikonversi ke string
    async with session.post(url, json={'query': query, 'variables': variables}) as response:
        return await response.json()

### Langkah 3: Membaca File NIM dan Menyiapkan Daftar Data
Langkah ini membaca file `nim.csv` yang berisi daftar NIM dan menyiapkan struktur data untuk menyimpan hasil unduhan.

In [ ]:
nim_list = pd.read_csv('../data/nim.csv')['nim'].tolist()
data_list = []
total = len(nim_list)
max_khs = 0
print(f"Total NIM: {total}")

### Langkah 4: Mengunduh Data secara asynchronousdan Memprosesnya 
Kita mengiterasi daftar NIM, mengunduh data untuk setiap NIM menggunakan fungsi `query_graphql`, dan memproses data tersebut untuk disimpan dalam daftar `data_list`. Juga, kita melacak jumlah maksimum entri KHS untuk mengatur kolom DataFrame nantinya.

In [ ]:
async def get_mahasiswa():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for nim in nim_list:
            tasks.append(query_graphql(session, nim))
        responses = await asyncio.gather(*tasks)
        for i, result in enumerate(responses):
            nim = nim_list[i]
            # print(f"Result for NIM {nim}: {result.get('data', {}).get('mahasiswa', {}).get('nama')}")
            mahasiswa = result.get('data', {}).get('mahasiswa', {})
            if mahasiswa:
                row = {
                    'kodeProdi': mahasiswa.get('kodeProdi'),
                    'angkatan': mahasiswa.get('angkatan'),
                    'semesterAwal': mahasiswa.get('semesterAwal'),
                    'nim': mahasiswa.get('nim'),
                    'nama': mahasiswa.get('nama'),
                    'jenisKelamin': mahasiswa.get('jenisKelamin'),
                    'tahunAkademikLulus': mahasiswa.get('tahunAkademikLulus'),
                    'tanggalLulus': mahasiswa.get('tanggalLulus'),
                    'lulus': mahasiswa.get('lulus'),
                    'masaStudi': mahasiswa.get('masaStudi'),
                }
                khs = mahasiswa.get('khs', [])
                global max_khs
                max_khs = max(max_khs, len(khs))
                for idx, k in enumerate(khs, start=1):
                    row.update({
                        f'khs{idx}_tahunAkademik': k.get('tahunAkademik'),
                        f'khs{idx}_ips': k.get('ips'),
                        f'khs{idx}_sksSmt': k.get('sksSmt'),
                        f'khs{idx}_ipk': k.get('ipk'),
                        f'khs{idx}_sksTotal': k.get('sksTotal'),
                        f'khs{idx}_statusMahasiswa': k.get('statusMahasiswa'),
                    })
                data_list.append(row)
                # print(f'Downloading data {i + 1}/{total} berhasil di download')
await get_mahasiswa()
print(f'{total} Data berhasil di download')

### Langkah 5: Membuat DataFrame dan Menyimpannya ke CSV
Kita membuat DataFrame dari daftar data yang telah diproses.

In [ ]:
df = pd.DataFrame(data_list)
columns = [
    'kodeProdi', 'angkatan', 'semesterAwal', 'nim', 'nama', 'jenisKelamin', 'tahunAkademikLulus', 'tanggalLulus', 'lulus', 'masaStudi'
] + [
    f'khs{idx}_{field}' for idx in range(1, max_khs + 1) for field in
    ['tahunAkademik', 'ips', 'sksSmt', 'ipk', 'sksTotal', 'statusMahasiswa']
]
df = df.reindex(columns=columns)


### Langkah 6: Memverifikasi Data
Langkah terakhir adalah memverifikasi beberapa baris pertama dari DataFrame untuk memastikan bahwa data telah diunduh dan diproses dengan benar.

In [ ]:
pd.set_option('display.max_columns', None)  # Menampilkan semua kolom
pd.set_option('display.expand_frame_repr', False)  # Menghindari pembungkusan frame
df.head(10)

# Validasi data yang sudah di download dengan NIM yang dicari

In [ ]:
# Validasi data yang sudah di download dengan NIM yang dicari
nim_yang_dicari = '105821100817'  # Ganti dengan NIM yang ingin Anda cari
data_nim = df.loc[df['nim'] == nim_yang_dicari]

# Menampilkan hasil pencarian dan cocokkan dengan KHS yang ada
if not data_nim.empty:
    for index, row in data_nim.iterrows():
        print(f"Data Mahasiswa dengan NIM: {row['nim']}")
        print("=================================")
        for column in data_nim.columns:
            print(f"{column}: {row[column]}")
else:
    print("Data dengan NIM tersebut tidak ditemukan.")